In [1]:
import cv2
import time
import numpy as np

In [17]:
class CaptureManager:
    def __init__(self, capture, previewWindowManager=None, shouldMirrorPreview=False):
        """
        Initializes the CaptureManager with a VideoCapture object and optional preview window manager.
        
        :param capture: OpenCV VideoCapture object
        :param previewWindowManager: Object responsible for showing preview window (optional)
        :param shouldMirrorPreview: Boolean indicating whether the preview should be mirrored
        """
        self.previewWindowManager = previewWindowManager
        self.shouldMirrorPreview = shouldMirrorPreview
        
        self._capture = capture
        self._channel = 0  # The default channel, for color (BGR) images
        self._enteredFrame = False
        self._frame = None
        self._imageFilename = None
        self._videoFilename = None
        self._videoEncoding = None
        self._videoWriter = None
        
        self._startTime = None
        self._framesElapsed = 0
        self._fpsEstimate = None

    @property
    def channel(self):
        return self._channel

    @channel.setter
    def channel(self, value):
        if self._channel != value:
            self._channel = value
            self._frame = None

    @property
    def frame(self):
        """Returns the current frame if it exists."""
        if self._enteredFrame and self._frame is None:
            self._frame = self.capture.retrieve()[1]
        return self._frame

    @property
    def isWritingImage(self):
        """Returns True if the capture manager is writing to an image file."""
        return self.imageFilename is not None

    @property
    def isWritingVideo(self):
        """Returns True if the capture manager is writing to a video file."""
        return self.videoFilename is not None

    def enterFrame(self):
        """
        Captures the next frame, if any.
        Ensures the previous frame was processed and exited before grabbing a new one.
        """
        assert not self._enteredFrame, "Previous enterFrame() had no matching exitFrame()"
        
        if self._capture is not None:
            self._enteredFrame = self._capture.grab()

    def exitFrame(self):
        """
        Draws to the window, writes to files, and releases the frame.
        """
        if self.frame is None:
            self._enteredFrame = False
            return

        # Update FPS estimate and related variables.
        if self._framesElapsed == 0:
            self._startTime = time.time()
        else:
            timeElapsed = time.time() - self._startTime
            self._fpsEstimate = self._framesElapsed / timeElapsed
        self._framesElapsed += 1
        
        # Draw to the window, if any.
        if self.previewWindowManager is not None:
            if self.shouldMirrorPreview:
                mirroredFrame = np.fliplr(self.frame).copy()
                self.previewWindowManager.show(mirroredFrame)
            else:
                self.previewWindowManager.show(self.frame)

        # Write to the image file, if any.
        if self.isWritingImage:
            cv2.imwrite(self.imageFilename, self.frame)
            self._imageFilename = None  # Reset image filename after writing.

        # Write to the video file, if any.
        self._writeVideoFrame()

        # Release the frame and reset the capture state.
        self._frame = None
        self._enteredFrame = False

    def writeImage(self, filename):
        """
        Write the next exited frame to an image file.
        """
        self._imageFilename = filename

    def startWritingVideo(self, filename, encoding=cv2.VideoWriter_fourcc(*'XVID')):
        """
        Start writing exited frames to a video file.

        :param filename: The name of the video file to write to
        :param encoding: The codec used for encoding the video
        """
        self._videoFilename = filename
        self._videoEncoding = encoding

    def stopWritingVideo(self):
        """
        Stop writing exited frames to a video file.
        """
        self._videoFilename = None
        self._videoEncoding = None
        if self._videoWriter is not None:
            self._videoWriter.release()  # Release the video writer
        self._videoWriter = None

    def _writeVideoFrame(self):
        """
        Write the current frame to a video file.
        """
        if not self._isWritingVideo:
            return

        if self._videoWriter is None:
            fps = self._capture.get(cv2.CAP_PROP_FPS)
            if fps == 0.0:  # If FPS is unknown, use an estimated FPS.
                if self._framesElapsed < 20:
                    return  # Wait for more frames to accumulate a stable FPS estimate.
                fps = self._fpsEstimate

            # Get video frame size.
            size = (int(self._capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
                    int(self._capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))

            # Initialize the video writer.
            self._videoWriter = cv2.VideoWriter(self._videoFilename, self._videoEncoding, fps, size)

        # Write the frame to the video.
        self._videoWriter.write(self.frame)



In [12]:
class WindowManager:
    def __init__(self, windowName, keypressCallback=None):
        """
        Initialize the WindowManager instance.

        :param windowName: Name of the window to create and manage.
        :param keypressCallback: Function to call when a key is pressed (optional).
        """
        self.keypressCallback = keypressCallback  # Function to handle keypress events
        self._windowName = windowName  # The window name
        self._isWindowCreated = False  # Flag indicating whether the window is created

    @property
    def isWindowCreated(self):
        """Return True if the window is created."""
        return self._isWindowCreated

    @isWindowCreated.setter
    def isWindowCreated(self, value):
        """Set the status of whether the window is created."""
        self._isWindowCreated = value

    def createWindow(self):
        """
        Create an OpenCV window with the specified window name.
        """
        cv2.namedWindow(self._windowName)
        self._isWindowCreated = True

    def show(self, frame):
        """
        Display the given frame in the window.

        :param frame: The frame to be displayed in the window.
        """
        if self._isWindowCreated:
            cv2.imshow(self._windowName, frame)

    def destroyWindow(self):
        """
        Destroy the OpenCV window associated with this WindowManager instance.
        """
        if self._isWindowCreated:
            cv2.destroyWindow(self._windowName)
            self._isWindowCreated = False

    def processEvents(self):
        """
        Process OpenCV window events. This checks for keypress events and triggers the callback
        if one exists.

        This method should be called regularly (e.g., in a main loop) to handle events.
        """
        keycode = cv2.waitKey(1)  # Wait for a key press for 1 ms
        if keycode != -1:  # If a key is pressed
            if self._keypressCallback is not None:
                # Discard any non-ASCII information encoded by keycode (e.g., extended codes).
                keycode = keycode & 0xFF  # Only keep the lower 8 bits (ASCII codes)
                self._keypressCallback(keycode)  # Call the key press callback


In [1]:
class Cameo:
    def __init__(self):
        """
        Initializes the Cameo application, setting up window and capture managers.
        """
        # Initialize the WindowManager with a callback to handle keypresses
        self._windowManager = WindowManager("Cameo", keypressCallback=self.onKeypress)
        
        # Initialize the CaptureManager to capture video from the camera
        # We pass True for shouldMirrorPreview to mirror the preview feed
        self._captureManager = CaptureManager(cv2.VideoCapture(0), self._windowManager, True)

    def run(self):
        """
        Run the main application loop, processing frames and events.
        """
        # Create the window for display
        self._windowManager.createWindow()

        # Main loop: process frames and events
        while self._windowManager.isWindowCreated:
            # Enter a new frame (grab a frame from the camera)
            self._captureManager.enterFrame()
            frame = self._captureManager.frame  # Retrieve the frame

            if frame is not None:
                # Process the frame here (e.g., apply a filter)
                processed_frame = self.process_frame(frame)

                # Update the frame in CaptureManager after processing
                self._captureManager._frame = processed_frame  # Directly set the processed frame
                
            # Exit the current frame and handle the window display logic
            self._captureManager.exitFrame()

            # Display the processed frame in the window
            self._windowManager.show(self._captureManager.frame)

            # Process any window events (like keypresses)
            self._windowManager.processEvents()

    def process_frame(self, frame):
        """
        Process the frame (apply filters like grayscale, blur, etc.).
        """
        # Convert to grayscale (simple filter)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian Blur to the grayscale image
        blurred_frame = cv2.GaussianBlur(gray_frame, (15, 15), 0)

        # You can add more filters or effects here if needed.
        # For example, edge detection:
        # edges = cv2.Canny(blurred_frame, 100, 200)

        # Return the processed frame (e.g., blurred or with effects)
        return blurred_frame

    def onKeypress(self, keycode):
        """
        Handle keypress events.

        - Space: Take a screenshot
        - Tab: Start/stop video recording (screencast)
        - Escape: Quit the application
        """
        if keycode == 32:  # Spacebar
            # Take a screenshot and save it as 'screenshot.png'
            self._captureManager.writeImage("screenshot.png")
        elif keycode == 9:  # Tab key
            # Start/stop video recording (screencast)
            if not self._captureManager.isWritingVideo:
                self._captureManager.startWritingVideo("screencast.avi")
            else:
                self._captureManager.stopWritingVideo()
        elif keycode == 27:  # Escape key
            # Quit the application
            self._windowManager.destroyWindow()

In [18]:
# Create and run the Cameo application
if __name__ == "__main__":
    app = Cameo()
    app.run()

AttributeError: 'CaptureManager' object has no attribute 'capture'